In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage import exposure
import os
import numpy as np
from skimage import io, color, feature, exposure
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix,roc_curve,auc,roc_auc_score
from sklearn.preprocessing import LabelEncoder
from skimage.transform import resize
import seaborn as sns


In [ ]:
# Function to extract HOG features from an image and visualize it
def extract_hog_features(image):
    # Resize the image to a smaller size (e.g., 128x128)
    resized_image = resize(image, (128, 128))

    # Convert the image to grayscale
    gray_image = color.rgb2gray(resized_image)

    # Calculate HOG features
    hog_features, hog_image = feature.hog(gray_image, visualize=True)

    # Enhance the contrast of the HOG image for better visualization
    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))

    return hog_features, hog_image_rescaled


In [ ]:
# Dataset Path
dataset_path = r"D:\ml project 1\ImageDataSet\dataset"

# List all subdirectories (assuming each subdirectory corresponds to a class)
class_folders = [f.path for f in os.scandir(dataset_path) if f.is_dir()]

# Lists to store features and corresponding labels
features_list = []
labels_list = []

# Loop through each class folder
for class_folder in class_folders:
    class_name = os.path.basename(class_folder)

    # Loop through each image in the class folder
    for image_filename in os.listdir(class_folder):
        image_path = os.path.join(class_folder, image_filename)

        # Load the image
        image = io.imread(image_path)

        # Extract HOG features and visualize
        hog_features, hog_image = extract_hog_features(image)

        # Display the original image and the HOG features
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4), sharex=True, sharey=True)

        ax1.axis('off')
        ax1.imshow(image, cmap=plt.cm.gray)
        ax1.set_title('Original Image')

        ax2.axis('off')
        ax2.imshow(hog_image, cmap=plt.cm.gray)
        ax2.set_title('HOG Features')

        plt.show()

        # Append HOG features to the features list
        features_list.append(hog_features)

        # Append the label to the labels list
        labels_list.append(class_name)

In [ ]:
# Convert lists to NumPy arrays
features_array = np.array(features_list)
labels_array = np.array(labels_list)
# Use LabelEncoder to convert class names into numeric labels
label_encoder = LabelEncoder()
numeric_labels = label_encoder.fit_transform(labels_array)
print(features_array)
print(numeric_labels)

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_array, numeric_labels, test_size=0.2, random_state=42)


In [ ]:
from sklearn.cluster import KMeans

# Use KMeans on training set
num_clusters = len(np.unique(y_train))
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(X_train)

# Assign labels to clusters based on majority class
cluster_labels = {}
for i in range(num_clusters):
    cluster_mask = (kmeans.labels_ == i)
    cluster_true_labels = y_train[cluster_mask]
    most_common_label = np.bincount(cluster_true_labels).argmax()
    cluster_labels[i] = most_common_label

# Use the trained K-means model on the test set
predicted_labels = []
for i in range(len(X_test)):
    # Extract features for the test image
    test_image = X_test[i].reshape(1, -1)

    # Use the trained K-means model to assign it to the closest cluster
    cluster_assignment = kmeans.predict(test_image)

    # Assign the label from the majority class in that cluster
    predicted_label = cluster_labels[cluster_assignment[0]]
    predicted_labels.append(predicted_label)

# Evaluate the performance
accuracy = accuracy_score(y_test, predicted_labels)
print(f"Accuracy of K-means with HOG features: {accuracy}")